In [1]:
from sklearn.datasets import fetch_openml #Loading the MNIST data
mnist = fetch_openml('mnist_784', version = 1)

In [2]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist['data'] , mnist['target']
print("Data shape : ", X.shape) #There are 70,000 images and each image has 784 features(28 X 28 pixels)
print("Target Shape: ", y.shape)

Data shape :  (70000, 784)
Target Shape:  (70000,)
